In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#https://docs.google.com/spreadsheets/d/1NfqXgq2l5vJw2M5ffCLv2zQsXy3iOqak6IAEe3U0AG4/edit?usp=sharing
    
sheet_id = '1NfqXgq2l5vJw2M5ffCLv2zQsXy3iOqak6IAEe3U0AG4'
    
raw_data = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

In [4]:
raw_data

,SEASON,GAME,BOWLING TEAM,BATTING TEAM,INNING,OVER,BOWLER,BATSMAN,BALL,RUNS,WICKET,WHO WAS OUT,DEFENSIVE PLAYER
0,Ball in Play League 1,Game 1,Team Baggage,We Got Ice,1,1,Jimmy O'Brien,Jack Doyle,1,6,NaN,NaN,NaN
1,Ball in Play League 1,Game 1,Team Baggage,We Got Ice,1,1,Jimmy O'Brien,Jack Doyle,2,1,NaN,NaN,NaN
2,Ball in Play League 1,Game 1,Team Baggage,We Got Ice,1,1,Jimmy O'Brien,Zo DeMalia,3,0,NaN,NaN,NaN
3,Ball in Play League 1,Game 1,Team Baggage,We Got Ice,1,1,Jimmy O'Brien,Zo DeMalia,4,1,NaN,NaN,NaN
4,Ball in Play League 1,Game 1,Team Baggage,We Got Ice,1,1,Jimmy O'Brien,Jack Doyle,5,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,Captain's League 2024,Game 2,The Breakdowns,The Story,3,1,Jimmy O'Brien,Jake Storiale,2,1,NaN,NaN,NaN
920,Captain's League 2024,Game 2,The Breakdowns,The Story,3,1,Jimmy O'Brien,Jack Sorensen,3,0,Caught,Jack Sorensen,Dalton Feely
921,Captain's League 2024,Game 2,The Breakdowns,The Story,3,1,Jimmy O'Brien,Dan Rourke,4,0,NaN,NaN,NaN
922,Captain's League 2024,Game 2,The Breakdowns,The Story,3,1,Jimmy O'Brien,Dan Rourke,5,0,Caught,Dan Rourke,Luke O'Brien


In [5]:
full_data = raw_data.copy(deep=True)

#full_data.loc[full_data['BATSMAN'] == 'Joez McFly', ['BATSMAN']] = 'Joez Mcfly'

full_data['DELIVERED'] = np.where(full_data['BALL'] != 'WIDE', 1, 0)
full_data['DOT'] = np.where((full_data['RUNS'] == 0) & 
                             (full_data['WICKET'].isna() == True), 1, 0)
full_data['WIDE BALLS'] = np.where(full_data['BALL'] == 'WIDE', 1, 0)
full_data['EXTRA RUNS'] = full_data['WIDE BALLS'] * full_data['RUNS']
full_data["4's"] = np.where(full_data['RUNS'] == 4, 1, 0)
full_data["6's"] = np.where(full_data['RUNS'] == 6, 1, 0)

full_data['INNINGS BATTED'] = full_data['GAME'] + full_data['INNING'].astype(str)

grouped_bowler_data = full_data.groupby(['SEASON', 'BOWLER']).agg({'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_bowler_data['OVERS'] = np.floor(grouped_bowler_data['DELIVERED'] / 6) \
                                + grouped_bowler_data['DELIVERED'] % 6 / 10

grouped_bowler_data['ECONOMY'] = round(grouped_bowler_data['RUNS'] \
                                    / grouped_bowler_data['OVERS'], 2)

overs = grouped_bowler_data.pop('OVERS')
grouped_bowler_data.insert = grouped_bowler_data.insert(1, overs.name, overs)
grouped_bowler_data = grouped_bowler_data.reset_index()
#grouped_bowler_data.reset_index(inplace = True)


grouped_batsman_data = full_data.groupby(['SEASON', 'BATSMAN']).agg({'GAME' : 'nunique',
                                                                     'INNINGS BATTED' : 'nunique',
                                                                     'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_batsman_data.rename(columns = {'DELIVERED' : 'BALLS FACED'}, inplace = True)
grouped_batsman_data['RUNS'] = grouped_batsman_data['RUNS'] - grouped_batsman_data['EXTRA RUNS']
grouped_batsman_data['STRIKE RATE'] = round(grouped_batsman_data['RUNS'] \
                                        / grouped_batsman_data['BALLS FACED'] * 100, 1)
grouped_batsman_data['AVG / INNINGS'] = round(grouped_batsman_data['RUNS'] \
                                        / grouped_batsman_data['INNINGS BATTED'], 2)
grouped_batsman_data = grouped_batsman_data.reset_index()

In [6]:
#full_data.loc[(full_data['SEASON'] == 'Ball in Play League 1') & (full_data['BOWLER'] == 'Jack Doyle') & (full_data['BALL'] == 'WIDE')]

full_data.loc[full_data['BATSMAN'] == 'Joez McFly', ['BATSMAN']] = 'Joez Mcfly'

In [7]:
grouped_bowler_data

,SEASON,BOWLER,DELIVERED,OVERS,RUNS,WICKET,DOT,WIDE BALLS,EXTRA RUNS,4's,6's,ECONOMY
0,Ball in Play League 1,Aaron Jones,30,5.0,81,2,2,3,3,7,5,16.20
1,Ball in Play League 1,Aatif Nawaz,12,2.0,42,0,2,4,6,5,2,21.00
2,Ball in Play League 1,BBD,42,7.0,77,8,10,3,4,6,5,11.00
3,Ball in Play League 1,Calvin Savage,12,2.0,44,1,3,5,5,2,5,22.00
4,Ball in Play League 1,Dalton Feely,6,1.0,20,0,2,0,0,2,2,20.00
5,Ball in Play League 1,Drew Davis,6,1.0,28,0,0,0,0,2,3,28.00
6,Ball in Play League 1,Jack Doyle,72,12.0,152,7,21,3,5,21,7,12.67
7,Ball in Play League 1,Jacobus Pienaar,53,8.5,137,3,17,10,15,13,9,16.12
8,Ball in Play League 1,Jake Storiale,24,4.0,82,0,4,1,2,6,8,20.50
9,Ball in Play League 1,Jaskaran Malhotra,74,12.2,149,8,15,8,15,20,4,12.21


In [8]:
grouped_batsman_data

,SEASON,BATSMAN,GAME,INNINGS BATTED,BALLS FACED,RUNS,WICKET,DOT,WIDE BALLS,EXTRA RUNS,4's,6's,STRIKE RATE,AVG / INNINGS
0,Ball in Play League 1,Aaron Jones,4,8,49,110,4,11,2,4,11,7,224.5,13.75
1,Ball in Play League 1,Aatif Nawaz,4,5,8,13,0,2,0,0,0,1,162.5,2.60
2,Ball in Play League 1,BBD,4,6,22,51,3,3,2,2,5,4,231.8,8.50
3,Ball in Play League 1,Calvin Savage,3,6,36,88,3,7,1,1,11,5,244.4,14.67
4,Ball in Play League 1,Dalton Feely,3,6,16,39,1,1,2,3,6,1,243.8,6.50
5,Ball in Play League 1,Drew Davis,4,7,43,98,4,7,4,4,13,4,227.9,14.00
6,Ball in Play League 1,Jack Doyle,4,6,32,84,2,4,6,9,9,5,262.5,14.00
7,Ball in Play League 1,Jacobus Pienaar,4,6,54,109,2,15,4,6,8,9,201.9,18.17
8,Ball in Play League 1,Jake Storiale,4,6,30,75,0,3,3,7,9,4,250.0,12.50
9,Ball in Play League 1,Jaskaran Malhotra,4,8,36,72,4,4,3,4,11,2,200.0,9.00


In [28]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_table
import pandas as pd

# Sample DataFrame for demonstration
df = grouped_bowler_data#.loc[:, ['SEASON', 'BOWLER', 'DELIVERED']]

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Ball in Play Analytics"),
    
#     # Bar chart
#     dcc.Graph(
#         id='bar-chart',
#         figure=px.bar(df, x='Category', y='Values', title='Bar Chart')
#     ),
    
    # Checkbox for an additional column
    html.Label("Select Season:"),
    dcc.Checklist(
        id='season-toggle',
        options=[
            {'label': col, 'value': col} for col in df['SEASON'].unique()
        ],
        value=df['SEASON'].unique().tolist()
    ),
    
    # Dropdown menu for multiple selections with "Select All" feature
    html.Label("Select Player(s):"),
    dcc.Dropdown(
        id='bowler-dropdown',
        options=[
            {'label': category, 'value': category} for category in df['BOWLER'].unique()
        ],
        value=df['BOWLER'].tolist(),  # Select all categories by default
        multi=True
    ),
    
    # Data table
    html.H3("Data Table"),
    dash_table.DataTable(
        id='data-table',
        columns=[
            {"name": i, "id": i} for i in df.columns
#             {'name': 'SEASON', 'id': 'SEASON'},
#             {'name': 'BOWLER', 'id': 'BOWLER'},
#             {'name': 'DELIVERED', 'id': 'DELIVERED'}
        ],
        data=df.to_dict('records'),
    )
])

# Define callback to update the bar chart and data table based on dropdown and checkbox selections
@app.callback(
    #[#Output('bar-chart', 'figure'),
     Output('data-table', 'data'),#],
    [Input('season-toggle', 'value'),
     Input('bowler-dropdown', 'value')]
)
def update_table(selected_season, selected_bowler):
    filtered_df = df[df['SEASON'].isin(selected_season) & (df['BOWLER'].isin(selected_bowler))]
    
#     # Update bar chart
#     fig = px.bar(filtered_df, x='Category', y='Values', title=f'Bar Chart for {", ".join(selected_categories)} ({", ".join(selected_column)})')
    
    # Update data table
    table_data = filtered_df.to_dict('records')
    
    return table_data

# Run the app
if __name__ == '__main__':
    #app.run_server(mode='inline', port=8050)
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [32]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import pandas as pd

# # Sample DataFrames for demonstration
# grouped_bowler_data = pd.DataFrame({
#     'SEASON': ['2020', '2020', '2021', '2021'],
#     'BOWLER': ['Player1', 'Player2', 'Player1', 'Player2'],
#     'DELIVERED': [100, 120, 110, 130]
# })

# grouped_batsman_data = pd.DataFrame({
#     'SEASON': ['2020', '2020', '2021', '2021'],
#     'BATSMAN': ['PlayerA', 'PlayerB', 'PlayerA', 'PlayerB'],
#     'RUNS': [50, 60, 55, 70]
# })

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app with tabs
app.layout = html.Div([
    html.H1("Ball in Play Analytics"),
    
    dcc.Tabs([
        # Tab for Bowler Data
        dcc.Tab(label='Bowler Data', children=[
            html.Label("Select Season:"),
            dcc.Checklist(
                id='season-toggle-bowler',
                options=[
                    {'label': col, 'value': col} for col in grouped_bowler_data['SEASON'].unique()
                ],
                value=[grouped_bowler_data['SEASON'].unique()[0]]
            ),
            
            html.Label("Select Bowler(s):"),
            dcc.Dropdown(
                id='bowler-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_bowler_data['BOWLER'].unique()
                ],
                value=grouped_bowler_data['BOWLER'].unique().tolist(),
                multi=True
            ),
            
            html.H3("Bowler Data Table"),
            dash_table.DataTable(
                id='data-table-bowler',
                columns=[
                    {"name": i, "id": i} for i in grouped_bowler_data.columns
#                     {'name': 'SEASON', 'id': 'SEASON'},
#                     {'name': 'BOWLER', 'id': 'BOWLER'},
#                     {'name': 'DELIVERED', 'id': 'DELIVERED'}
                ],
                data=grouped_bowler_data.to_dict('records'),
            )
        ]),
        
        # Tab for Batsman Data
        dcc.Tab(label='Batsman Data', children=[
            html.Label("Select Season:"),
            dcc.Checklist(
                id='season-toggle-batsman',
                options=[
                    {'label': col, 'value': col} for col in grouped_batsman_data['SEASON'].unique()
                ],
                value = grouped_batsman_data['SEASON'].unique()
            ),
            
            html.Label("Select Batsman(s):"),
            dcc.Dropdown(
                id='batsman-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_batsman_data['BATSMAN'].unique()
                ],
                value=grouped_batsman_data['BATSMAN'].unique(), #.tolist(),
                multi=True
            ),
            
            html.H3("Batsman Data Table"),
            dash_table.DataTable(
                id='data-table-batsman',
                columns=[
                    {"name": i, "id": i} for i in grouped_batsman_data.columns
#                     {'name': 'SEASON', 'id': 'SEASON'},
#                     {'name': 'BATSMAN', 'id': 'BATSMAN'},
#                     {'name': 'RUNS', 'id': 'RUNS'}
                ],
                data=grouped_batsman_data.to_dict('records'),
            )
        ])
    ])
])

# Callback for updating Bowler Data Table
@app.callback(
    Output('data-table-bowler', 'data'),
    [Input('season-toggle-bowler', 'value'),
     Input('bowler-dropdown', 'value')]
)
def update_data_table_bowler(selected_seasons, selected_bowlers):
    filtered_df = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_seasons) & grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    table_data = filtered_df.to_dict('records')
    return table_data

# Callback for updating Batsman Data Table
@app.callback(
    Output('data-table-batsman', 'data'),
    [Input('season-toggle-batsman', 'value'),
     Input('batsman-dropdown', 'value')]
)
def update_data_table_batsman(selected_seasons, selected_batsmen):
    filtered_df = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_seasons) & grouped_batsman_data['BATSMAN'].isin(selected_batsmen)]
    table_data = filtered_df.to_dict('records')
    return table_data

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [52]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import plotly.express as px
import pandas as pd
import numpy as np
import math


sheet_id = '1NfqXgq2l5vJw2M5ffCLv2zQsXy3iOqak6IAEe3U0AG4'
    
raw_data = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

full_data = raw_data.copy(deep=True)

#full_data.loc[full_data['BATSMAN'] == 'Joez McFly', ['BATSMAN']] = 'Joez Mcfly'

full_data['DELIVERED'] = np.where(full_data['BALL'] != 'WIDE', 1, 0)
full_data['DOT'] = np.where((full_data['RUNS'] == 0) & 
                             (full_data['WICKET'].isna() == True), 1, 0)
full_data['WIDE BALLS'] = np.where(full_data['BALL'] == 'WIDE', 1, 0)
full_data['EXTRA RUNS'] = full_data['WIDE BALLS'] * full_data['RUNS']
full_data["4's"] = np.where(full_data['RUNS'] == 4, 1, 0)
full_data["6's"] = np.where(full_data['RUNS'] == 6, 1, 0)

full_data['INNINGS BATTED'] = full_data['GAME'] + full_data['INNING'].astype(str)

grouped_bowler_data = full_data.groupby(['SEASON', 'BOWLER']).agg({'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_bowler_data['OVERS'] = np.floor(grouped_bowler_data['DELIVERED'] / 6) \
                                    + grouped_bowler_data['DELIVERED'] % 6 / 10

grouped_bowler_data['ECONOMY'] = round(grouped_bowler_data['RUNS'] \
                                    / grouped_bowler_data['OVERS'], 2)
overs = grouped_bowler_data.pop('OVERS')
grouped_bowler_data.insert = grouped_bowler_data.insert(1, overs.name, overs)
grouped_bowler_data = grouped_bowler_data.reset_index()
#grouped_bowler_data.reset_index(inplace = True)
    
grouped_batsman_data = full_data.groupby(['SEASON', 'BATSMAN']).agg({'GAME' : 'nunique',
                                                                     'INNINGS BATTED' : 'nunique',
                                                                     'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_batsman_data.rename(columns = {'DELIVERED' : 'BALLS FACED'}, inplace = True)
grouped_batsman_data['RUNS'] = grouped_batsman_data['RUNS'] \
                                            - grouped_batsman_data['EXTRA RUNS']
grouped_batsman_data['STRIKE RATE'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['BALLS FACED'] * 100, 1)
grouped_batsman_data['AVG / INNINGS'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['INNINGS BATTED'], 2)
grouped_batsman_data = grouped_batsman_data.reset_index()


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app with tabs
app.layout = html.Div([
    html.H1("Ball in Play Analytics", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 20}),
    
    dcc.Tabs([
        # Tab for Bowler Data
        dcc.Tab(label='Bowler Data',
                style={'fontSize': 20, 'color': '#4CAF50'},
                selected_style={'fontSize': 20, 'color': '#4CAF50', 'fontWeight': 'bold'},
                children=[
            html.Label("Select Season:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Checklist(
                id='season-toggle-bowler',
                options=[
                    {'label': col, 'value': col} for col in grouped_bowler_data['SEASON'].unique()
                ],
                value=grouped_bowler_data['SEASON'].unique(),
                style={'marginBottom': 20, 'color': '#333'}
            ),
            
            html.Label("Select Bowler(s):", style={'fontSize': 16, 'color': '#333'}),
            dcc.Checklist(
                id='bowler-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_bowler_data['BOWLER'].unique()
                ],
                value=grouped_bowler_data['BOWLER'].unique().tolist(),
                #multi=True,
                #inline = True, 
                style={'marginBottom': 20}
            ),
            
            html.H3("Bowler Data Table", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 10}),
            dash_table.DataTable(
                id='data-table-bowler',
                columns=[
                    {"name": i, "id": i} for i in grouped_bowler_data.columns
                ],
                data=grouped_bowler_data.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '2px solid #4CAF50', 'boxShadow': '0px 0px 10px #888888'},
                style_header={'backgroundColor': '#4CAF50', 'color': 'white', 'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                style_cell={'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
            ),
            
            html.Div(style={'marginBottom': 40}),
                    
            # Dropdown for Y-axis selection
            html.Label("Select Y-axis:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Dropdown(
                id='y-axis-dropdown-bowler',
                options=[
                    #{'label': 'DELIVERED', 'value': 'DELIVERED'}
                    {'label': col, 'value': col} for col in grouped_bowler_data.columns
                ],
                value='DELIVERED',
                style={'marginBottom': 20}
            ),
                    
            # Bar chart
            dcc.Graph(
                id='bar-chart-bowler',
                style={'height': '400px'},
            )
        ]),
        
        # Tab for Batsman Data
        dcc.Tab(label='Batsman Data',
                style={'fontSize': 20, 'color': '#4CAF50'},
                selected_style={'fontSize': 20, 'color': '#4CAF50', 'fontWeight': 'bold'},
                children=[
            html.Label("Select Season:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Checklist(
                id='season-toggle-batsman',
                options=[
                    {'label': col, 'value': col} for col in grouped_batsman_data['SEASON'].unique()
                ],
                value=grouped_batsman_data['SEASON'].unique(),
                style={'marginBottom': 20, 'color': '#333'}
            ),
            
            html.Label("Select Batsman(s):", style={'fontSize': 16, 'color': '#333'}),
            dcc.Dropdown(
                id='batsman-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_batsman_data['BATSMAN'].unique()
                ],
                value=grouped_batsman_data['BATSMAN'].unique().tolist(),
                multi=True,
                style={'marginBottom': 20}
            ),
            
            html.H3("Batsman Data Table", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 10}),
            dash_table.DataTable(
                id='data-table-batsman',
                columns=[
                    {"name": i, "id": i} for i in grouped_batsman_data.columns
                ],
                data=grouped_batsman_data.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '2px solid #4CAF50', 'boxShadow': '0px 0px 10px #888888'},
                style_header={'backgroundColor': '#4CAF50', 'color': 'white', 'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                style_cell={'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
            ),
                    
            html.Div(style={'marginBottom': 40}),
                    
            # Dropdown for Y-axis selection
            html.Label("Select Y-axis:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Dropdown(
                id='y-axis-dropdown-batsman',
                options=[
                    {'label': col, 'value': col} for col in grouped_batsman_data.columns
                ],
                value='INNINGS BATTED',
                style={'marginBottom': 20}
            ),
                    
            # Bar chart
            dcc.Graph(
                id='bar-chart-batsman',
                style={'height': '400px'},
            )
        ])
    ])
])


# Callback for updating player selection list based on selected season for Bowler Data
@app.callback(
    Output('bowler-dropdown', 'options'),
    [Input('season-toggle-bowler', 'value')]
)
def update_bowler_dropdown(selected_season):
    available_bowlers = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_season)]['BOWLER'].unique()
    options = [{'label': bowler, 'value': bowler} for bowler in available_bowlers]
    return options


# Callback for updating player selection list based on selected season for Batsman Data
@app.callback(
    Output('batsman-dropdown', 'options'),
    [Input('season-toggle-batsman', 'value')]
)
def update_batsman_dropdown(selected_season):
    available_batsmen = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_season)]['BATSMAN'].unique()
    options = [{'label': batsman, 'value': batsman} for batsman in available_batsmen]
    return options


# Callback for updating bar chart based on selected options for Bowler Data
@app.callback(
    Output('bar-chart-bowler', 'figure'),
    [Input('season-toggle-bowler', 'value'),
     Input('bowler-dropdown', 'value'),
     Input('y-axis-dropdown-bowler', 'value')]
)
def update_barchart_bowler(selected_seasons, selected_bowlers, y_axis_column):
    #filtered_df = grouped_bowler_data[grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    filtered_df = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_seasons) & grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    fig = px.bar(filtered_df,
                 x='BOWLER',
                 y=y_axis_column,
                 title=f'Bowler Data - {y_axis_column}',
                 color = 'SEASON')
    return fig


# Callback for updating bar chart based on selected options for Batsman Data
@app.callback(
    Output('bar-chart-batsman', 'figure'),
    [Input('season-toggle-batsman', 'value'),
     Input('batsman-dropdown', 'value'),
     Input('y-axis-dropdown-batsman', 'value')]
)
def update_barchart_batsman(selected_seasons, selected_batsman, y_axis_column):
    filtered_df = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_seasons) & grouped_batsman_data['BATSMAN'].isin(selected_batsman)]
    fig = px.bar(filtered_df,
                 x='BATSMAN',
                 y=y_axis_column,
                 title=f'Batsman Data - {y_axis_column}',
                 color = 'SEASON')
    return fig


# Callback for updating Bowler Data Table
@app.callback(
    Output('data-table-bowler', 'data'),
    [Input('season-toggle-bowler', 'value'),
     Input('bowler-dropdown', 'value')]
)
def update_data_table_bowler(selected_seasons, selected_bowlers):
    
    grouped_bowler_data = full_data.groupby(['SEASON', 'BOWLER']).agg({'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

    grouped_bowler_data['OVERS'] = np.floor(grouped_bowler_data['DELIVERED'] / 6) \
                                    + grouped_bowler_data['DELIVERED'] % 6 / 10

    grouped_bowler_data['ECONOMY'] = round(grouped_bowler_data['RUNS'] \
                                    / grouped_bowler_data['OVERS'], 2)
    overs = grouped_bowler_data.pop('OVERS')
    grouped_bowler_data.insert = grouped_bowler_data.insert(1, overs.name, overs)
    grouped_bowler_data = grouped_bowler_data.reset_index()
    #grouped_bowler_data.reset_index(inplace = True)


    filtered_df = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_seasons) & grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    table_data = filtered_df.to_dict('records')
    return table_data


# Callback for updating Batsman Data Table
@app.callback(
    Output('data-table-batsman', 'data'),
    [Input('season-toggle-batsman', 'value'),
     Input('batsman-dropdown', 'value')]
)
def update_data_table_batsman(selected_seasons, selected_batsmen):
    
    grouped_batsman_data = full_data.groupby(['SEASON', 'BATSMAN']).agg({'GAME' : 'nunique',
                                                                     'INNINGS BATTED' : 'nunique',
                                                                     'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

    grouped_batsman_data.rename(columns = {'DELIVERED' : 'BALLS FACED'}, inplace = True)
    grouped_batsman_data['RUNS'] = grouped_batsman_data['RUNS'] - grouped_batsman_data['EXTRA RUNS']
    grouped_batsman_data['STRIKE RATE'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['BALLS FACED'] * 100, 1)
    grouped_batsman_data['AVG / INNINGS'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['INNINGS BATTED'], 2)
    grouped_batsman_data = grouped_batsman_data.reset_index()

    
    filtered_df = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_seasons) & grouped_batsman_data['BATSMAN'].isin(selected_batsmen)]
    table_data = filtered_df.to_dict('records')
    return table_data

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [93]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import plotly.express as px
import pandas as pd
import numpy as np
import math


sheet_id = '1NfqXgq2l5vJw2M5ffCLv2zQsXy3iOqak6IAEe3U0AG4'
    
raw_data = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")

full_data = raw_data.copy(deep=True)

#full_data.loc[full_data['BATSMAN'] == 'Joez McFly', ['BATSMAN']] = 'Joez Mcfly'

full_data['DELIVERED'] = np.where(full_data['BALL'] != 'WIDE', 1, 0)
full_data['DOT'] = np.where((full_data['RUNS'] == 0) & 
                             (full_data['WICKET'].isna() == True), 1, 0)
full_data['WIDE BALLS'] = np.where(full_data['BALL'] == 'WIDE', 1, 0)
full_data['EXTRA RUNS'] = full_data['WIDE BALLS'] * full_data['RUNS']
full_data["4's"] = np.where(full_data['RUNS'] == 4, 1, 0)
full_data["6's"] = np.where(full_data['RUNS'] == 6, 1, 0)

full_data['INNINGS BATTED'] = full_data['GAME'] + full_data['INNING'].astype(str)

grouped_bowler_data = full_data.groupby(['SEASON', 'BOWLER']).agg({'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_bowler_data['OVERS'] = np.floor(grouped_bowler_data['DELIVERED'] / 6) \
                                    + grouped_bowler_data['DELIVERED'] % 6 / 10

grouped_bowler_data['ECONOMY'] = round(grouped_bowler_data['RUNS'] \
                                    / grouped_bowler_data['OVERS'], 2)
overs = grouped_bowler_data.pop('OVERS')
grouped_bowler_data.insert = grouped_bowler_data.insert(1, overs.name, overs)
grouped_bowler_data = grouped_bowler_data.reset_index()
#grouped_bowler_data.reset_index(inplace = True)
    
grouped_batsman_data = full_data.groupby(['SEASON', 'BATSMAN']).agg({'GAME' : 'nunique',
                                                                     'INNINGS BATTED' : 'nunique',
                                                                     'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

grouped_batsman_data.rename(columns = {'DELIVERED' : 'BALLS FACED',
                                           'GAME' : 'GAMES'}, inplace = True)
grouped_batsman_data['RUNS'] = grouped_batsman_data['RUNS'] \
                                            - grouped_batsman_data['EXTRA RUNS']
grouped_batsman_data['STRIKE RATE'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['BALLS FACED'] * 100, 1)
grouped_batsman_data['AVG / INNINGS'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['INNINGS BATTED'], 2)
grouped_batsman_data = grouped_batsman_data.reset_index()


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app with tabs
app.layout = html.Div([
    html.H1("Ball in Play Dashboard", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 20, 'font-family': 'Arial, sans-serif'}),
    
    dcc.Tabs([
        # Tab for Bowler Data
        dcc.Tab(label='Bowler Data',
                style={'fontSize': 20, 'color': '#4CAF50'},
                selected_style={'fontSize': 20, 'color': '#4CAF50', 'fontWeight': 'bold'},
                children=[
            html.Label("Select Season:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Checklist(
                id='season-toggle-bowler',
                options=[
                    {'label': col, 'value': col} for col in grouped_bowler_data['SEASON'].unique()
                ],
                value=grouped_bowler_data['SEASON'].unique(),
                style={'marginBottom': 20, 'color': '#333'}
            ),
            
            html.Label("Select Bowler(s):", style={'fontSize': 16, 'color': '#333'}),
            html.Button('Select All', id = 'select-all-bowlers', n_clicks = 0, style = {'fintSize': 16, 'marginBottom': 20}),
            html.Button('Deselect All', id = 'deselect-all-bowlers', n_clicks = 0, style = {'fintSize': 16, 'marginBottom': 20}),        
            dcc.Checklist(
                id='bowler-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_bowler_data['BOWLER'].unique()
                ],
                value=grouped_bowler_data['BOWLER'].unique().tolist(),
                #multi=True,
                #inline = True, 
                style={'marginBottom': 20, 'columnCount': 4}
            ),
            
            html.H3("Bowler Data Table", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 10}),
            dash_table.DataTable(
                id='data-table-bowler',
                columns=[
                    {"name": i, "id": i} for i in grouped_bowler_data.columns
                ],
                data=grouped_bowler_data.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '2px solid #4CAF50', 'boxShadow': '0px 0px 10px #888888'},
                style_header={'backgroundColor': '#4CAF50', 'color': 'white', 'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                style_cell={'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                sort_action='native',  # 'native' enables sorting on the client side
                sort_mode='multi',  # 'multi' allows sorting by multiple columns
            ),
            
            html.Div(style={'marginBottom': 40}),
                    
            # Dropdown for Y-axis selection
            html.Label("Select Y-axis:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Dropdown(
                id='y-axis-dropdown-bowler',
                options=[
                    #{'label': 'DELIVERED', 'value': 'DELIVERED'}
                    #{'label': col, 'value': col} for col in grouped_bowler_data.columns
                    {'label': col, 'value': col} for col in grouped_bowler_data.columns if col not in ['SEASON', 'BOWLER']
                ],
                value='DELIVERED',
                style={'marginBottom': 20}
            ),
                    
            # Bar chart
            dcc.Graph(
                id='bar-chart-bowler',
                style={'height': '400px'},
            )
        ]),
        
        # Tab for Batsman Data
        dcc.Tab(label='Batsman Data',
                style={'fontSize': 20, 'color': '#4CAF50'},
                selected_style={'fontSize': 20, 'color': '#4CAF50', 'fontWeight': 'bold'},
                children=[
            html.Label("Select Season:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Checklist(
                id='season-toggle-batsman',
                options=[
                    {'label': col, 'value': col} for col in grouped_batsman_data['SEASON'].unique()
                ],
                value=grouped_batsman_data['SEASON'].unique(),
                style={'marginBottom': 20, 'color': '#333'}
            ),
            
            html.Label("Select Batsman(s):", style={'fontSize': 16, 'color': '#333'}),
            html.Button('Select All', id = 'select-all-batsman', n_clicks = 0, style = {'fintSize': 16, 'marginBottom': 20}),
            html.Button('Deselect All', id = 'deselect-all-batsman', n_clicks = 0, style = {'fintSize': 16, 'marginBottom': 20}), 
            dcc.Checklist(
                id='batsman-dropdown',
                options=[
                    {'label': category, 'value': category} for category in grouped_batsman_data['BATSMAN'].unique()
                ],
                value=grouped_batsman_data['BATSMAN'].unique().tolist(),
                #multi=True,
                #inline = True,
                style={'marginBottom': 20, 'columnCount': 4}
            ),
            
            html.H3("Batsman Data Table", style={'textAlign': 'center', 'color': '#4CAF50', 'marginBottom': 10}),
            dash_table.DataTable(
                id='data-table-batsman',
                columns=[
                    {"name": i, "id": i} for i in grouped_batsman_data.columns
                ],
                data=grouped_batsman_data.to_dict('records'),
                style_table={'overflowX': 'auto', 'border': '2px solid #4CAF50', 'boxShadow': '0px 0px 10px #888888'},
                style_header={'backgroundColor': '#4CAF50', 'color': 'white', 'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                style_cell={'textAlign': 'center', 'font-family': 'Arial, sans-serif'},
                sort_action='native',  # 'native' enables sorting on the client side
                sort_mode='multi',  # 'multi' allows sorting by multiple columns
            ),
                    
            html.Div(style={'marginBottom': 40}),
                    
            # Dropdown for Y-axis selection
            html.Label("Select Y-axis:", style={'fontSize': 16, 'color': '#333'}),
            dcc.Dropdown(
                id='y-axis-dropdown-batsman',
                options=[
                    #{'label': col, 'value': col} for col in grouped_batsman_data.columns
                    {'label': col, 'value': col} for col in grouped_batsman_data.columns if col not in ['SEASON', 'BATSMAN']
                ],
                value='INNINGS BATTED',
                style={'marginBottom': 20}
            ),
                    
            # Bar chart
            dcc.Graph(
                id='bar-chart-batsman',
                style={'height': '400px'},
            )
        ])
    ])
], style={'font-family': 'Arial, sans-serif'})


# Callback for updating player selection list based on selected season for Bowler Data
@app.callback(
    Output('bowler-dropdown', 'options'),
    [Input('season-toggle-bowler', 'value')]
)
def update_bowler_dropdown(selected_season):
    available_bowlers = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_season)]['BOWLER'].unique()
    options = [{'label': bowler, 'value': bowler} for bowler in available_bowlers]
    return options

# Callback to update bowler dropdown options when "Select All" or "Deselect All" button is clicked
@app.callback(
    Output('bowler-dropdown', 'value'),
    [Input('select-all-bowlers', 'n_clicks'),
     Input('deselect-all-bowlers', 'n_clicks')], 
    prevent_initial_call=True
)
def update_bowler_values(n_clicks_select_all, n_clicks_deselect_all):
    # Use dash.callback_context to distinguish triggers
    triggered_input = dash.callback_context.triggered_id
    
    if triggered_input == 'select-all-bowlers':
        # Return all bowlers when the button is clicked
        if n_clicks_select_all is not None and n_clicks_select_all > 0:
            return grouped_bowler_data['BOWLER'].unique().tolist()
        else:
            # Default value (no change)
            return dash.no_update

    elif triggered_input == 'deselect-all-bowlers':
        # Return all bowlers when the button is clicked
        if n_clicks_deselect_all is not None and n_clicks_deselect_all > 0:
            return [] #grouped_bowler_data['BOWLER'].unique().tolist()
        else:
            # Default value (no change)
            return dash.no_update

    else:
        # Default value (no change)
        #raise dash.exceptions.PreventUpdate
        return dash.no_update



    
    
    
# Callback for updating player selection list based on selected season for Batsman Data
@app.callback(
    Output('batsman-dropdown', 'options'),
    [Input('season-toggle-batsman', 'value')]
)
def update_batsman_dropdown(selected_season):
    available_batsmen = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_season)]['BATSMAN'].unique()
    options = [{'label': batsman, 'value': batsman} for batsman in available_batsmen]
    return options


# Callback to update batsman dropdown options when "Select All" or "Deselect All" button is clicked
@app.callback(
    Output('batsman-dropdown', 'value'),
    [Input('select-all-batsman', 'n_clicks'),
     Input('deselect-all-batsman', 'n_clicks')], 
    prevent_initial_call=True
)
def update_batsman_values(n_clicks_select_all, n_clicks_deselect_all):
    # Use dash.callback_context to distinguish triggers
    triggered_input = dash.callback_context.triggered_id
    
    if triggered_input == 'select-all-batsman':
        # Return all batsman when the button is clicked
        if n_clicks_select_all is not None and n_clicks_select_all > 0:
            return grouped_batsman_data['BATSMAN'].unique().tolist()
        else:
            # Default value (no change)
            return dash.no_update

    elif triggered_input == 'deselect-all-batsman':
        # Return all bowlers when the button is clicked
        if n_clicks_deselect_all is not None and n_clicks_deselect_all > 0:
            return []
        else:
            # Default value (no change)
            return dash.no_update

    else:
        # Default value (no change)
        #raise dash.exceptions.PreventUpdate
        return dash.no_update




# Callback for updating bar chart based on selected options for Bowler Data
@app.callback(
    Output('bar-chart-bowler', 'figure'),
    [Input('season-toggle-bowler', 'value'),
     Input('bowler-dropdown', 'value'),
     Input('y-axis-dropdown-bowler', 'value')]
)
def update_barchart_bowler(selected_seasons, selected_bowlers, y_axis_column):
    #filtered_df = grouped_bowler_data[grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    filtered_df = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_seasons) & grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    fig = px.bar(filtered_df,
                 x='BOWLER',
                 y=y_axis_column,
                 title=f'Bowler Data - {y_axis_column}',
                 color = 'SEASON')
    return fig


# Callback for updating bar chart based on selected options for Batsman Data
@app.callback(
    Output('bar-chart-batsman', 'figure'),
    [Input('season-toggle-batsman', 'value'),
     Input('batsman-dropdown', 'value'),
     Input('y-axis-dropdown-batsman', 'value')]
)
def update_barchart_batsman(selected_seasons, selected_batsman, y_axis_column):
    filtered_df = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_seasons) & grouped_batsman_data['BATSMAN'].isin(selected_batsman)]
    fig = px.bar(filtered_df,
                 x='BATSMAN',
                 y=y_axis_column,
                 title=f'Batsman Data - {y_axis_column}',
                 color = 'SEASON')
    return fig


# Callback for updating Bowler Data Table
@app.callback(
    Output('data-table-bowler', 'data'),
    [Input('season-toggle-bowler', 'value'),
     Input('bowler-dropdown', 'value')]
)
def update_data_table_bowler(selected_seasons, selected_bowlers):
    
    grouped_bowler_data = full_data.groupby(['SEASON', 'BOWLER']).agg({'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

    grouped_bowler_data['OVERS'] = np.floor(grouped_bowler_data['DELIVERED'] / 6) \
                                    + grouped_bowler_data['DELIVERED'] % 6 / 10

    grouped_bowler_data['ECONOMY'] = round(grouped_bowler_data['RUNS'] \
                                    / grouped_bowler_data['OVERS'], 2)
    overs = grouped_bowler_data.pop('OVERS')
    grouped_bowler_data.insert = grouped_bowler_data.insert(1, overs.name, overs)
    grouped_bowler_data = grouped_bowler_data.reset_index()
    #grouped_bowler_data.reset_index(inplace = True)


    filtered_df = grouped_bowler_data[grouped_bowler_data['SEASON'].isin(selected_seasons) & grouped_bowler_data['BOWLER'].isin(selected_bowlers)]
    table_data = filtered_df.to_dict('records')
    return table_data


# Callback for updating Batsman Data Table
@app.callback(
    Output('data-table-batsman', 'data'),
    [Input('season-toggle-batsman', 'value'),
     Input('batsman-dropdown', 'value')]
)
def update_data_table_batsman(selected_seasons, selected_batsmen):
    
    grouped_batsman_data = full_data.groupby(['SEASON', 'BATSMAN']).agg({'GAME' : 'nunique',
                                                                     'INNINGS BATTED' : 'nunique',
                                                                     'DELIVERED' : 'sum',
                                                                   'RUNS' : 'sum',
                                                                   'WICKET' : 'count',
                                                                   'DOT' : 'sum',
                                                                   'WIDE BALLS' : 'sum',
                                                                   'EXTRA RUNS' : 'sum',
                                                                   "4's" : 'sum',
                                                                   "6's" : 'sum'
                                                          })

    grouped_batsman_data.rename(columns = {'DELIVERED' : 'BALLS FACED',
                                           'GAME' : 'GAMES'}, inplace = True)
    grouped_batsman_data['RUNS'] = grouped_batsman_data['RUNS'] - grouped_batsman_data['EXTRA RUNS']
    grouped_batsman_data['STRIKE RATE'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['BALLS FACED'] * 100, 1)
    grouped_batsman_data['AVG / INNINGS'] = round(grouped_batsman_data['RUNS'] \
                                            / grouped_batsman_data['INNINGS BATTED'], 2)
    grouped_batsman_data = grouped_batsman_data.reset_index()

    
    filtered_df = grouped_batsman_data[grouped_batsman_data['SEASON'].isin(selected_seasons) & grouped_batsman_data['BATSMAN'].isin(selected_batsmen)]
    table_data = filtered_df.to_dict('records')
    return table_data

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
